In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from experiment import experiment
from experiment import test_loss
import random


scaler = MinMaxScaler(feature_range=(-1, 1))


#loading sonar data
sonar_data = pd.read_csv("sonar.csv").to_numpy()
sonar_x = sonar_data[:,:-1]
sonar_x = scaler.fit_transform(sonar_x)
#add intercept
bias = np.ones((sonar_data.shape[0],1))
sonar_x = np.hstack((bias, sonar_x)).astype(float)
sonar_y = sonar_data[:,-1]
#convert labels to +1 -1
sonar_y = np.where(sonar_y == "M",1,-1)


p = sonar_x.shape[0]

normal_iters, quant_iters,  w_quant, w_quant_prev, w_quant_prev2, w = experiment(sonar_x, sonar_y,1,p)

"""print(f"sonar normal iterations: {np.mean(normal_iters)}, quantized iterations: {np.mean(quant_iters)}, "
      f"normal loss {np.mean(normal_loss)}, quantized loss {np.mean(quant_loss)}, quantized iters {quant_iters}")"""
got_same_w = (np.sign(w) == w_quant)

# a bunch of diagnostic outputs
print(got_same_w)
print(w)
print(w_quant)
print(quant_iters,normal_iters)
print(test_loss(np.sign(w), sonar_x.T,sonar_y.T))
print(test_loss(w, sonar_x.T,sonar_y.T))
print(test_loss(w_quant, sonar_x.T,sonar_y.T))


in loop
[[ True]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [ True]
 [ True]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [False]
 [False]
 [False]
 [ True]
 [ True]
 [ True]
 [ True]
 [False]
 [False]
 [ True]
 [False]
 [ True]
 [ True]
 [False]
 [False]
 [ True]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [ True]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [ True]
 [ True]
 [False]
 [False]
 [ True]]
[[ 3.91154261]
 [ 1.2696732 ]
 [ 0.21815129]
 [-1.59962031]
 [ 0.44652279]
 [ 0.59887052]
 [-0.64157358]
 [-0.91438508]
 [-1.2228932 ]
 [ 1.12874928]
 [ 0.48051162]
 [ 1.99290778]
 [ 1.95475957]
 [ 0.5096081 ]
 [-1.1360824 ]
 [-0.17500569]
 [-0.82157123]
 [-0.31320642]
 [-0.27020271]
 [ 0.32804385]
 [ 0.61648105]
 [ 0.06551609]
 [ 0.77953737]
 [ 0.40645657]
 [ 1.80054502]
 [-0.96089384]
 [-0.0258013 ]
 [-0.50898782]
 [ 0.80412668]
 [ 0.21

In [2]:
#check loop behavior

loop = np.hstack((w_quant, w_quant_prev, w_quant_prev2))
print(loop)

# it seems like almost all of the bits are being flipped every time, which is not expected 
# given the w* in the previous output.
# I thought that any feature with w* not in [-1,1] would not be flipped repeatedly, of which there are quite a few

[[ 1. -1.  1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [-1. -1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]]


In [3]:
print(np.hstack((w_quant_prev , w_quant, np.sign(w))))

# the w's that we are stuck looping through are not np.sign(w)

# I think that this is due to the random init of w, and that this behavior
# can be seen in 2 dimensions as well.

[[-1.  1.  1.]
 [ 1. -1.  1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1.  1.]
 [ 1.  1.  1.]
 [ 1.  1. -1.]
 [-1.  1.  1.]
 [-1.  1.  1.]
 [-1.  1.  1.]
 [-1.  1.  1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1. -1.]
 [-1.  1.  1.]
 [-1.  1. -1.]
 [-1.  1.  1.]
 [-1. -1. -1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1. -1.]
 [ 1. -1.  1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1. -1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1. -1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1.  1.]
 [ 1. -1.  1.]
 [ 1. -1. -1.]]


In [4]:
# comparing actual gradient to quantized gradient for w*

from QuantLog import quantlogistic
from NormalLog import normallogistic
quant_loss, quant_gradient = quantlogistic(np.sign(w),sonar_x.T,sonar_y.T)
normal_loss, normal_gradient = normallogistic(np.sign(w),sonar_x.T,sonar_y.T)

print(np.hstack((quant_gradient,normal_gradient)))

# the gradient is close, but the sign is not always preserved 

[[-0.30676329 -0.26158379]
 [ 0.17379191  0.14448501]
 [ 0.20356014  0.16874904]
 [ 0.22150171  0.18494393]
 [ 0.22905281  0.19249102]
 [ 0.19481905  0.16224272]
 [ 0.14806354  0.1205706 ]
 [ 0.09412055  0.07985025]
 [ 0.11901688  0.10135632]
 [ 0.13626329  0.11375859]
 [ 0.11505467  0.09612846]
 [ 0.09834682  0.07960135]
 [ 0.07936467  0.06066084]
 [ 0.07066791  0.05023992]
 [ 0.12728151  0.10033259]
 [ 0.09403087  0.07357391]
 [ 0.07071793  0.0548034 ]
 [ 0.05751625  0.04501018]
 [ 0.02881561  0.02195713]
 [-0.00671023 -0.00970616]
 [-0.05143035 -0.04656039]
 [-0.0881449  -0.07754422]
 [-0.10061827 -0.08772867]
 [-0.10312093 -0.08957719]
 [-0.12664443 -0.11134076]
 [-0.13368278 -0.11725256]
 [-0.14484373 -0.12556618]
 [-0.16135971 -0.13887113]
 [-0.15027904 -0.12535706]
 [-0.10488719 -0.08487957]
 [-0.03304061 -0.02274921]
 [ 0.04603378  0.04385089]
 [ 0.08819491  0.07885445]
 [ 0.12539309  0.11144712]
 [ 0.12517351  0.11002881]
 [ 0.14147445  0.12553178]
 [ 0.14943704  0.13191877]
 

In [5]:
#get the quantized gradient of the final w_quant

loss_quant_w, gradient_quant_w = quantlogistic(w_quant,sonar_x.T,sonar_y.T)

vis_stationary_pts = np.hstack((w_quant,gradient_quant_w, w))
print(vis_stationary_pts)


# reason - I think that if any of the values in w* are between -1 and 1, the gradient will bounce back and forth between
# the quantized feasible points. 

# problem - why is the quant gradient at w_quant point towards w* in some instances



[[ 1.00000000e+00  4.54106280e-01  3.99876282e+00]
 [-1.00000000e+00 -3.18063928e-01  1.83419517e+00]
 [-1.00000000e+00 -3.41688564e-01 -4.02787789e-01]
 [-1.00000000e+00 -3.52034889e-01 -1.59465715e+00]
 [-1.00000000e+00 -3.76873617e-01  9.75436675e-01]
 [-1.00000000e+00 -3.27598226e-01  2.11900301e-03]
 [-1.00000000e+00 -2.45451135e-01 -1.11503931e-02]
 [-1.00000000e+00 -1.85470126e-01 -1.26712670e+00]
 [-1.00000000e+00 -2.34398053e-01 -1.39458929e+00]
 [-1.00000000e+00 -2.79788335e-01  1.44214173e+00]
 [-1.00000000e+00 -2.62664321e-01  6.60521524e-01]
 [-1.00000000e+00 -2.66798450e-01  1.55393330e+00]
 [-1.00000000e+00 -2.31034541e-01  1.91279139e+00]
 [-1.00000000e+00 -1.84353723e-01  5.45175569e-01]
 [-1.00000000e+00 -2.24671309e-01 -7.61755469e-01]
 [-1.00000000e+00 -1.74165032e-01 -2.88014531e-01]
 [-1.00000000e+00 -1.18552672e-01 -5.49289433e-01]
 [-1.00000000e+00 -9.15636887e-02 -1.15105125e+00]
 [-1.00000000e+00 -6.09423427e-02  1.65618161e-01]
 [ 1.00000000e+00 -5.18899327e-

In [18]:
# line search: iterate along the line between w_quant and w_wuant prev to try to find
# the best w in that search path

from QuantLog import test_loss

ts = np.linspace(0,1,100)
loss = []
ws =[]
for t in ts:
    w_current = t*w_quant + (1-t)*w_quant_prev
    loss.append(test_loss(w_current, sonar_x.T, sonar_y.T))
    ws.append(w_current)

best_loss = np.min(loss)
best_w = ws[np.argmin(loss)]

print(best_loss,best_w)
    





# is not super great

0.3719806763285024 [[ 0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [ 1.        ]
 [ 1.        ]
 [ 0.01010101]
 [ 0.01010101]
 [ 0.01010101]
 [ 0.01010101]
 [ 0.01010101]
 [ 0.01010101]
 [ 0.01010101]
 [ 0.01010101]
 [ 0.01010101]
 [ 0.01010101]
 [-1.        ]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]
 [-0.01010101]]
